In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
from tqdm import tqdm, tqdm_notebook
import xgboost as xgb
import os
import lightgbm as lgb
from catboost import CatBoostClassifier
import seaborn as sns
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error, roc_auc_score, cohen_kappa_score
from sklearn.linear_model import LogisticRegressionCV
import time
import gc
from gensim.models import Word2Vec
%matplotlib inline
import embedder

/data/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/data/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
A = pd.read_csv(r"~/Deep-Learning-Project-Template/data/train.csv", dtype={'PetID': str})
# Add some features and decipher categorical columns

# Cat/Dog
A['Type'] = A['Type'].apply(lambda x: 'Dog' if x == 1 else 'Cat')

# binary noname col
A['Name'] = A['Name'].fillna('Unnamed')
A['No_name'] = 0
A.loc[A['Name'] == 'Unnamed', 'No_name'] = 1

A['binned_age'] = pd.cut(A['Age'], bins=[-1,3,6,12,24,36,10000])

# Meaningless names - 2 characters and less (maybe 3 as well)
# A['meaningless_name'] = 0
# A.loc[len(A['Name']) <= 2, 'meaningless_name'] = 1

# is pure-bred
A['Pure_breed'] = 0
A.loc[A['Breed2'] == 0, 'Pure_breed'] = 1

A['health'] = A['Vaccinated'].astype(str) + '_' + \
                A['Dewormed'].astype(str) + '_' + \
                A['Sterilized'].astype(str) + '_' + \
                A['Health'].astype(str)
            
A['Free'] = A['Fee'].apply(lambda x: 1 if x == 0 else 0)


A['Description'] = A['Description'].fillna('')
A['desc_length'] = A['Description'].apply(lambda x: len(x))
A['desc_words'] = A['Description'].apply(lambda x: len(x.split()))
A['averate_word_length'] = A['desc_length'] / A['desc_words']
A.loc[~np.isfinite(A['averate_word_length']), 'averate_word_length'] = 0

A = A.drop(columns=['Unnamed: 0'])

In [3]:
A.head()

,Type,Name,Age,Breed1,Breed2,Gender,Color1,Color2,Color3,MaturitySize,...,BinaryLabel,set,No_name,binned_age,Pure_breed,health,Free,desc_length,desc_words,averate_word_length
0,Cat,Nibble,3,299,0,1,1,7,0,1,...,1,0.0,0,"(-1, 3]",1,2_2_2_1,0,359,69,5.202899
1,Cat,No Name Yet,1,265,0,1,1,2,0,2,...,1,0.0,0,"(-1, 3]",1,3_3_3_1,1,118,23,5.130435
2,Dog,Brisco,1,307,0,1,2,7,0,2,...,1,0.0,0,"(-1, 3]",1,1_1_2_1,1,393,69,5.695652
3,Dog,Miko,4,307,0,2,1,2,0,2,...,1,1.0,0,"(3, 6]",1,1_1_2_1,0,146,25,5.840000
4,Dog,Hunter,1,307,0,1,1,0,0,2,...,1,1.0,0,"(-1, 3]",1,2_2_2_1,1,390,81,4.814815


In [4]:
def add_col_name_to_val(df, rel_cols):
    for c in rel_cols:
        df[c] = c + "_" + df[c].astype(str)
    return df

In [5]:
real_cat_cols = ['Type', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed', 'binned_age',
                 'Sterilized', 'Health', 'State']

In [11]:
from category_encoders import OrdinalEncoder

for_learning_encs = A.loc[(A['set'] == 0) & (A['Quantity'] == 1)]
y = for_learning_encs['BinaryLabel']

cols_to_drop = ['Description', 'Name', 'PetID', 'RescuerID', 'health']

X = for_learning_encs.drop(columns=cols_to_drop+['BinaryLabel', 'AdoptionSpeed', 'binned_age', 'set', 'Quantity'])
real_cat_cols = ['Type', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
                 'Sterilized', 'Health', 'State']
# enc = OrdinalEncoder(cols=real_cat_cols).fit(X, y)
# X = enc.transform(X)

In [12]:
cat_sz = [(col, X[col].unique().shape[0]) for col in X.columns
              if col in real_cat_cols]

# cat_vars = embedder.preprocessing.categorize(A)
embedding_dict = embedder.preprocessing.pick_emb_dim(cat_sz, max_dim=50)
X_encoded, encoders = embedder.preprocessing.encode_categorical(X)
import keras
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=10,
                           mode='min',
                           verbose=1)

embedder_instance = embedder.classification.Embedder(embedding_dict, model_json=None)
embedder_instance_res = embedder_instance.fit(X_encoded, y, epochs=15, early_stop=early_stop)

Train on 7401 samples, validate on 1851 samples
Epoch 1/15
7401/7401 [==============================] - 2s 247us/step - loss: 4.0144 - acc: 0.7086 - val_loss: 3.7199 - val_acc: 0.5570
Epoch 2/15
7401/7401 [==============================] - 0s 40us/step - loss: 4.2769 - acc: 0.7233 - val_loss: 4.2117 - val_acc: 0.7358
Epoch 3/15
7401/7401 [==============================] - 0s 39us/step - loss: 4.2952 - acc: 0.7306 - val_loss: 4.2117 - val_acc: 0.7358
Epoch 4/15
7401/7401 [==============================] - 0s 39us/step - loss: 4.2952 - acc: 0.7306 - val_loss: 4.2117 - val_acc: 0.7358
Epoch 5/15
7401/7401 [==============================] - 0s 39us/step - loss: 4.2952 - acc: 0.7306 - val_loss: 4.2117 - val_acc: 0.7358
Epoch 6/15
7401/7401 [==============================] - 0s 40us/step - loss: 4.2952 - acc: 0.7306 - val_loss: 4.2117 - val_acc: 0.7358
Epoch 7/15
7401/7401 [==============================] - 0s 40us/step - loss: 4.2952 - acc: 0.7306 - val_loss: 4.2117 - val_acc: 0.7358
Epoch 

In [13]:
cols_to_drop = ['Description', 'Name', 'PetID', 'RescuerID', 'health']
real_cat_cols = ['Type', 'Breed1', 'Breed2', 'Gender', 'Color1', 'Color2',
       'Color3', 'MaturitySize', 'FurLength', 'Vaccinated', 'Dewormed',
                 'Sterilized', 'Health', 'State']

X = A.drop(columns=cols_to_drop+['BinaryLabel', 'AdoptionSpeed', 'binned_age', 'set', 'Quantity'])
X_encoded, encoders = embedder.preprocessing.encode_categorical(X)
b = embedder_instance_res.transform(X_encoded, as_df=True)


In [14]:
df_concat = pd.concat([b.drop(columns=['Age', 'Fee', 'VideoAmt', 'PhotoAmt', 'No_name', 'Pure_breed', 'Free', 'desc_length', 'desc_words', 'averate_word_length'])
                       , A.drop(columns=real_cat_cols+['Description', 'Name', 'RescuerID', 'health'])], axis=1)

In [15]:
df_concat.head()

,Type_0,Color1_0,Color1_1,Color1_2,Color1_3,Health_0,Health_1,Breed2_0,Breed2_1,Breed2_2,...,AdoptionSpeed,BinaryLabel,set,No_name,binned_age,Pure_breed,Free,desc_length,desc_words,averate_word_length
0,0.011917,0.053246,0.039660,-0.021724,0.032977,0.033117,0.022611,-20.687265,4.603824,17.525082,...,2,1,0.0,0,"(-1, 3]",1,0,359,69,5.202899
1,0.011917,0.053246,0.039660,-0.021724,0.032977,0.033117,0.022611,-20.687265,4.603824,17.525082,...,0,1,0.0,0,"(-1, 3]",1,1,118,23,5.130435
2,-0.046664,-0.013779,-0.032106,-0.040567,0.029062,0.033117,0.022611,-20.687265,4.603824,17.525082,...,3,1,0.0,0,"(-1, 3]",1,1,393,69,5.695652
3,-0.046664,0.053246,0.039660,-0.021724,0.032977,0.033117,0.022611,-20.687265,4.603824,17.525082,...,2,1,1.0,0,"(3, 6]",1,0,146,25,5.840000
4,-0.046664,0.053246,0.039660,-0.021724,0.032977,0.033117,0.022611,-20.687265,4.603824,17.525082,...,2,1,1.0,0,"(-1, 3]",1,1,390,81,4.814815


In [16]:
print(list(df_concat.columns))

['Type_0', 'Color1_0', 'Color1_1', 'Color1_2', 'Color1_3', 'Health_0', 'Health_1', 'Breed2_0', 'Breed2_1', 'Breed2_2', 'Breed2_3', 'Breed2_4', 'Breed2_5', 'Breed2_6', 'Breed2_7', 'Breed2_8', 'Breed2_9', 'Breed2_10', 'Breed2_11', 'Breed2_12', 'Breed2_13', 'Breed2_14', 'Breed2_15', 'Breed2_16', 'Breed2_17', 'Breed2_18', 'Breed2_19', 'Breed2_20', 'Breed2_21', 'Breed2_22', 'Breed2_23', 'Breed2_24', 'Breed2_25', 'Breed2_26', 'Breed2_27', 'Breed2_28', 'Breed2_29', 'Breed2_30', 'Breed2_31', 'Breed2_32', 'Breed2_33', 'Breed2_34', 'Breed2_35', 'Breed2_36', 'Breed2_37', 'Breed2_38', 'Breed2_39', 'Breed2_40', 'Breed2_41', 'Breed2_42', 'Breed2_43', 'Breed2_44', 'Breed2_45', 'Breed2_46', 'Breed2_47', 'Breed2_48', 'Breed2_49', 'Gender_0', 'Color2_0', 'Color2_1', 'Color2_2', 'Color2_3', 'Sterilized_0', 'Sterilized_1', 'Vaccinated_0', 'Vaccinated_1', 'FurLength_0', 'FurLength_1', 'Breed1_0', 'Breed1_1', 'Breed1_2', 'Breed1_3', 'Breed1_4', 'Breed1_5', 'Breed1_6', 'Breed1_7', 'Breed1_8', 'Breed1_9', 'Br

In [17]:
train = df_concat.loc[df_concat['set'] == 0]
y_train = train['BinaryLabel']
train = train.drop(columns=["PetID", "Quantity", "AdoptionSpeed", "BinaryLabel", "set", "binned_age"])
test = df_concat.loc[df_concat['set'] == 1]
y_test = test['BinaryLabel']
test = test.drop(columns=["PetID", "Quantity", "AdoptionSpeed", "BinaryLabel", "set", "binned_age"])

In [18]:
p_adopted = 0.73
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

from sklearn.tree import DecisionTreeClassifier
dtc = DecisionTreeClassifier(min_samples_leaf=100)
dtc = dtc.fit(X=train, y=y)
cur_pred = dtc.predict(train)
rand_pred = np.random.choice([0, 1], size=cur_pred.size, p=[1-p_adopted, p_adopted])
# rand_pred = np.random.rand(cur_pred.size)
print("Train MSE: {}".format(sklearn.metrics.mean_squared_error(y_train, cur_pred)))
print("Train Random MSE: {}\n".format(sklearn.metrics.mean_squared_error(y_train, rand_pred)))
print("Train logloss: {}".format(sklearn.metrics.log_loss(y_train, sigmoid(cur_pred))))
print("Train Random logloss: {}\n".format(sklearn.metrics.log_loss(y_train, sigmoid(rand_pred))))

cur_pred = dtc.predict(test)
rand_pred = np.random.choice([0, 1], size=cur_pred.size, p=[1-p_adopted, p_adopted])
# rand_pred = np.random.rand(cur_pred.size)
print("Test MSE: {}".format(sklearn.metrics.mean_squared_error(y_test, cur_pred)))
print("Test Random MSE: {}\n".format(sklearn.metrics.mean_squared_error(y_test, rand_pred)))
print("Test logloss: {}".format(sklearn.metrics.log_loss(y_test, sigmoid(cur_pred))))
print("Test Random logloss: {}".format(sklearn.metrics.log_loss(y_test, sigmoid(rand_pred))))

Train MSE: 0.24535235624729787
Train Random MSE: 0.38737570255079984

Train logloss: 0.5545589968737161
Train Random logloss: 0.6094723141426025

Test MSE: 0.2546476437527021
Test Random MSE: 0.3804582792909641

Test logloss: 0.5556175384137113
Test Random logloss: 0.6002953629090181


In [19]:
D = df_concat.drop(columns=["PetID", "Quantity", "AdoptionSpeed", "BinaryLabel", "set", "binned_age"])
D = D.to_numpy()
vec_dir = r"/data/home/Shai/Deep-Learning-Project-Template/data/cat_embedded_vectors"
for i, vec in tqdm_notebook(enumerate(D)):
    np.save(os.path.join(vec_dir, df_concat.iloc[i]['PetID']+".npy"), vec)
vec.shape

FileNotFoundError: [Errno 2] No such file or directory: '~/Deep-Learning-Project-Template/data/cat_embedded_vectors/86e1089a3.npy'